In [2]:
import numpy as np

# Dense (Slow)
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense

# Sparse (Fast)
from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.core.sparse.hamiltonian_big import hamiltonian_general_sparse

from quant_rotor.models.dense.support_ham import H_kinetic, H_potential, basis_m_to_p_matrix_conversion, write_matrix_elements

In [15]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(threshold=np.inf)

In [ ]:
def H_kinetic_new(states: int, sites: int, K: np.ndarray) -> np.ndarray:
    """
    Constructs the dense kinetic energy Hamiltonian in a many-body tensor-product space,
    using a single-particle operator K applied independently to each site.

    Parameters
    ----------
    state : int
        Total number states in the system, counting the ground state. Ex: system of -1, 0, 1 would be a system of 3 states.
    site : int
        The number of rotors (sites) in the system.
    K : np.ndarray
        Dense Kinetic energy matrix in the p-basis, shape (state, state).

    Returns
    -------
    np.ndarray
        Dense many-body kinetic Hamiltonian.
    """

    # Create a matrix of the shape of Kinetic energy Hamiltonian filled with zeros.
    K_H = np.zeros((states**sites, states**sites), dtype=complex)

    for x in range(sites):

        # Define the total number of elements in the matrix operator, which represent the left and right sites that are not interacting
        # by n_lambda and n_mu respectively.
        n_lambda = states**(x)
        n_mu = states**(sites - x - 1)

        # Iterate through all elements of the Kinetic energy matrix operator.
        for p in range(states):
            for p_prime in range(states):

                # Extract an associated element.
                val = K[p, p_prime]

                # Check if element is non zero.
                if val == 0:
                    continue  # skip writing 0s

                for Lambda in range(int(n_lambda)):
                    for mu in range(int(n_mu)):

                        # Calculate the indices in the hamiltonian.
                        i = mu + p * n_mu + Lambda * states * n_mu
                        j = mu + p_prime * n_mu + Lambda * states * n_mu

                        # Assign a values to associated.
                        K_H[i, j] += val
    return K_H

In [18]:
site = 3
state = 5
g = 1

In [19]:
K, V = write_matrix_elements((state - 1) // 2)

V_tensor = V.reshape(state, state, state, state)  # Adjust if needed

h_full = basis_m_to_p_matrix_conversion(K, state)
v_full = basis_m_to_p_matrix_conversion(V_tensor, state)

v_full = (v_full * g).reshape(state**2, state**2)

In [20]:
v_full.real

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  ,  0.  ,  0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  , -0.25, -0.25,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0